# LLM examples

Learning goals: 

- Practice different introductory use cases of LLMs

## Setup

- To run this notebook you will need to create an account in cohere.ai, and get your API Key.
- You also need to install the python package `cohere`.

## Why cohere? 

- It was one of the first providers of LLMs as a service, was already in the market before ChatGPT. 

- Easy to use Python SDK. 

- A free account gives you access to a Trial API key, which allows 5 calls per minute. 

- Similar in structure to other APIs and frameworks (if you learn this, you'll navigate the others easily). 

## Preparing the work

Let's initialize the environment:

In [1]:
import cohere

In [2]:
# Log in to cohere, navigate to DASHBOARD (at the top), then to API Keys (at the sidebar)

# Never store your key in the code or push it to your repository!!!

with open("cohere.key") as f:
    COHERE_API_KEY = f.read()
cohere_client = cohere.ClientV2(COHERE_API_KEY)

## Use case 1: Prompt templates

### Prompt templates from scratch

LLM APIs allow us to incorporate conversational interactions in our applications to replace or complement traditional code. 

For example, we can call a LLM to classify the language of a text, instead of writing the code of a ML language classifier. Or we can ask an LLM to extract relevant fields from a text, instead of using a form with predefined fields.

The basic mechanism to do this is to create a prompt template, which is a structured way of asking the LLM for a specific task. In this notebook, we will learn patterns to create prompt templates, and we will see how to use them in different use cases.

For example, let's say we want to classify the language of a text. Let's use a very simple approach:

In [3]:
simple_prompt_template = "Return the language of the text between triple quotation marks: ```!text!``` " 

user_text = input("Enter some text")
final_prompt = simple_prompt_template.replace("!text!", user_text)

formatted_message = {
    'role': 'user',
    'content': final_prompt 
}

response = cohere_client.chat(
    model="command-a-03-2025",
    messages=[formatted_message],
)

language = response.message.content[0].text
print(language)

The language of the text between triple quotation marks is **Catalan**.


In [4]:
response.dict()

{'id': '6b1dee43-80fa-45d5-bf95-8ac4f68b8229',
 'finish_reason': 'COMPLETE',
 'message': {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'The language of the text between triple quotation marks is **Catalan**.'}]},
 'usage': {'billed_units': {'input_tokens': 21.0, 'output_tokens': 14.0},
  'tokens': {'input_tokens': 517.0, 'output_tokens': 17.0},
  'cached_tokens': 448.0}}

This is good but has a few problems. The LLM will try to be helpful and give us a long answer, instead of just the name of the language. Also, you might need to specify additional instructions, such as the list of languages to choose from, or what to do if the language is not in the list.

A common, well-established pattern to create prompt templates is the following:

1. Define the task and the input. 
2. Define the output format. 
3. Provide examples.
4. Add any additional instructions or constraints. 




In [5]:
user_text = input("Enter some text")

messages = [
    {
        "role": "system",
        "content": """
You are a language identification system.

Task:
Detect the primary natural language of the user-provided text.

Instructions:
- The user input will appear between triple backticks.
- Identify the dominant language of the text.
- Respond with exactly ONE word in UPPERCASE.
- Do NOT include punctuation, explanations, or additional text.
- If the language cannot be determined confidently, respond with: UNKNOWN
- If the text contains multiple languages, respond with the dominant one.

Example:
Input:
```Hello, how are you?```
Output:
ENGLISH
"""
    },
    {
        "role": "user",
        "content": f"""
Input:
```{user_text}```
Output:
"""
    }
]


response = cohere_client.chat(
    messages = messages,
    model='command-a-03-2025',
    temperature=0.0
)

response.message.content[0].text

'CATALAN'

### Extension: In-prompt examples

We can use LLMs to "simulate" the way AI systems work and provide "examples" of what we want to achieve, as part of the prompt. 

This is know as "few-shot", "in-prompt" learning. 

The example below tries to classify the restaurant type based on its name. It does so by preparing a "template" of known inputs and outputs, then asking to complete a last input: 

In [6]:
user_text = input("Enter some text")
messages = [
    {
        "role": "system",
        "content": """
You are a system for customer support in banking.
Your first task is to detect the part of the request that refers to all details 
of a specific transaction and add the tags [TRANSACTION] and [/TRANSACTION] around it.
Detais include: date, amount, location, and merchant. 
The user input will appear between triple backticks.
If there is no transaction, just return the original text without any tags.
Examples: 
Input: ```Hello, good morning. I have a problem with a payment of 5.60 EUR made in Starbucks yesterday. Thanks!```
Segmentation: Hello, good morning. I have a problem with a payment of [TRANSACTION]5.60 EUR made in Starbucks yesterday[/TRANSACTION]. Thanks!
Input: ```What is this??? I don't recognize a payment of 1200 EUR in Amazon on the 5th of May.```
Segmentation: What is this??? I don't recognize a payment of [TRANSACTION]1200 EUR in Amazon on the 5th of May[/TRANSACTION].
Input: ```Hi I want information on mortgage loans. Also, I want to cancel a direct debit of 50 EUR in the gym, I don't have time to go there!```
Segmentation: Hi I want information on mortgage loans. Also, I want to cancel a [TRANSACTION]direct debit of 50 EUR in the gym[/TRANSACTION], I don't have time to go there!
Input: ```I want to know the balance of my account.```
Segmentation: I want to know the balance of my account.
Input: ```How much is the comission for a new credit card?```
Segmentation: How much is the comission for a new credit card?
"""
    },
    {
        "role": "user",
        "content": f"""Input:
```{user_text}```
Segmentation:
"""
    }
]

In [7]:
response = cohere_client.chat(
    messages = messages,
    model='command-a-03-2025',
    temperature=0.0,
    max_tokens=200
)

response.message.content[0].text

'Hi, I am a customer of yours. The insurance Zurich charged [TRANSACTION]100 euros more than expected. Direct debit with data 23/2/2026[/TRANSACTION]. Can you help me with this?'

Mind the cost increases with the number of examples. You can get info of the costs:

In [8]:
response.dict()

{'id': '09a13233-003e-4ba2-b8f2-c783b66822c8',
 'finish_reason': 'COMPLETE',
 'message': {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'Hi, I am a customer of yours. The insurance Zurich charged [TRANSACTION]100 euros more than expected. Direct debit with data 23/2/2026[/TRANSACTION]. Can you help me with this?'}]},
 'usage': {'billed_units': {'input_tokens': 427.0, 'output_tokens': 48.0},
  'tokens': {'input_tokens': 956.0, 'output_tokens': 51.0},
  'cached_tokens': 480.0}}

## Use case: LLMs as user interface

Natural language can be the new interface. 

The example below asks the user to write a text specifying whether to turn the lights on or off.  


In [9]:
from ipywidgets import widgets

checkbox = widgets.Checkbox(
    value=False,
    description='Lights on',
    disabled=False,
    indent=False
)
checkbox

Checkbox(value=False, description='Lights on', indent=False)

In [11]:
user_input = input("Enter your message")
response = cohere_client.chat(
    messages = [{
        'role': 'system',
        'content': """
The user will write a text about turning off or on the lights.
The user might use different words to describe the action or expressions, 
like turn on, turn off, activate, deactivate, switch on, switch off, etc.
The user might also use different words to describe the lights,
like lights, lamps, bulbs, etc.
Return a single word: "true" if the user wants to turn on the lights, and "false" if the user wants to turn them off.
        """
    },
    {
        'role': 'user',
        'content': user_input
    }
    ],
    model = 'command-a-03-2025',
)

outcome = response.message.content[0].text
print("(Just for debugging, the output text is", outcome, ")")

if outcome == "true":
    checkbox.value = True
elif outcome == "false":
    checkbox.value = False
else:
    print("Unexpected outcome")

(Just for debugging, the output text is true )


## Use case: Structuring data

Use LLMs to extract information and add it to a certain template. 

The next example simulates a text from a health record and how the system would extract the necessary information into a template. 

In [13]:
system_prompt = """
The user will write a text simulating an electronic health record.
Extract the information necessary to populate this json object:
{
  "age": X,
  "temperature": X,
  "reason for visit": "X",
  "tests performed": ["X", "X", "X"],
    "diagnosis": "X"
}
If the information is not present, return "Not found".
Respond only with the json object. 
"""

user_input = input("Enter your message")

response = cohere_client.chat(
    messages = [
      {
        'role': 'system',
        'content': system_prompt  
      }, 
      {
          'role': 'user',
          'content': user_input
      }
    ],
    model='command-a-03-2025',
)

print(response.message.content[0].text)

```json
{
  "age": "Not found",
  "temperature": 28,
  "reason for visit": "Not found",
  "tests performed": ["blood test"],
  "diagnosis": "Not found"
}
```


## Bear in mind: chat history

This example illustrates the use of a chatbot, where the intent to be fulfilled is accomplished after several messages, and each message needs to include the previous chat history. 

Let's do it in the wrong way first: 

In [14]:
message1 = {'role': 'user', 'content': 'Who discovered polonium and radium?'}
message2 = {'role': 'user', 'content': 'When was she born?'}

response1 = cohere_client.chat(messages=[message1], model='command-a-03-2025')
response2 = cohere_client.chat(messages=[message2], model='command-a-03-2025')

print(response2.message.content[0].text)

I'm sorry, but I don't have enough information to answer your question. Could you please provide the name of the person you are referring to? I'll be happy to help you find the birth date if you give me the necessary details.


In [15]:
response1.message

AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text="Polonium and radium were discovered by **Marie Curie** and **Pierre Curie**, a husband-and-wife team of scientists, in the late 19th century. Specifically:\n\n1. **Polonium** (element 84) was discovered in 1898 by Marie and Pierre Curie. They named it after Marie's homeland, Poland (Latin: *Polonia*).\n\n2. **Radium** (element 88) was also discovered by Marie and Pierre Curie in 1898, while they were studying pitchblende, a uranium-rich mineral. They isolated radium and recognized its highly radioactive properties.\n\nMarie Curie's work on radioactivity, including the discovery of polonium and radium, earned her the Nobel Prize in Physics in 1903 (shared with Pierre Curie and Henri Becquerel) and the Nobel Prize in Chemistry in 1911.")], citations=None)

We need to add the previous chat history, that was stored already in the response json:

In [16]:
response = cohere_client.chat(
    messages=[
        message1,
        response1.message,
        message2
    ],
    model='command-a-03-2025',
)
print(response.message.content[0].text)


Marie Curie was born on **November 7, 1867**, in Warsaw, Poland (then part of the Russian Empire). She is widely regarded as one of the most influential scientists in history, known for her pioneering work on radioactivity and her discoveries of polonium and radium.


In [17]:
response1.message

AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text="Polonium and radium were discovered by **Marie Curie** and **Pierre Curie**, a husband-and-wife team of scientists, in the late 19th century. Specifically:\n\n1. **Polonium** (element 84) was discovered in 1898 by Marie and Pierre Curie. They named it after Marie's homeland, Poland (Latin: *Polonia*).\n\n2. **Radium** (element 88) was also discovered by Marie and Pierre Curie in 1898, while they were studying pitchblende, a uranium-rich mineral. They isolated radium and recognized its highly radioactive properties.\n\nMarie Curie's work on radioactivity, including the discovery of polonium and radium, earned her the Nobel Prize in Physics in 1903 (shared with Pierre Curie and Henri Becquerel) and the Nobel Prize in Chemistry in 1911.")], citations=None)